In [ ]:
import git
import pandas as pd
import json
import os
import pymysql as sql

#Cloning git hub repository
repo_url = "https://github.com/PhonePe/pulse.git"
destination_path = r"C:\Users\Kumar\Desktop\Guvi\PhonePe Transaction Insights"

git.Repo.clone_from(repo_url, destination_path)
print("Repository cloned successfully!")


In [21]:
def format_state_name(State):
    return State.replace('-', ' ').title()

#Connecting python to Mysql
myconnection = sql.connect(host="127.0.0.1",user="root",password="Selva765799#")
cursor=myconnection.cursor()
cursor.execute("create database if not exists PhonePe_Project")
cursor.execute("use PhonePe_project")

#Define a Function to craete table from dataframe
def create_table_from_dataframe(df, table_name): 

    dtype_mapping={
        'int64':'INTEGER',
        'float64':'FLOAT',
        'object':'TEXT',
        'bool':'BOOLEAN',
        'datetime64[ns]':'DATETIME',
    }    
    
    columns = ",".join(F"{col} {dtype_mapping[str(dtype)]}" for col,dtype in zip(df.columns,df.dtypes))
    
    cursor.execute(f"create table if not exists PhonePe_Project.{table_name} ({columns})")


#Define a Function to Insert Data into Tables

def insert_data_from_dataframe(df, table_name):
        insert_query = f"INSERT IGNORE INTO PhonePe_Project.{table_name} ({','.join(df.columns)}) VALUES ({','.join(['%s'] * len(df.columns))})"
        # Convert DataFrame to list of tuples for bulk insert
        data = [tuple(i) for i in df.values]
        
        # Bulk insert data into MySQL
        cursor.executemany(insert_query, data)
        myconnection.commit()

#Getting Aggregated transactions dataframe from json file and loading it into mysql database as table

path = r"C:\Users\Kumar\Desktop\Guvi\PhonePe Transaction Insights\data\aggregated\transaction\country\india\state"
Agg_trans_list=os.listdir(path)

clm={'State':[], 'Year':[],'Quarter':[],'Transaction_type':[], 'Transaction_count':[], 'Transaction_amount':[]}
    
for i in Agg_trans_list:
    p_i=os.path.join(path, i)
    Agg_yr=os.listdir(p_i)
    for j in Agg_yr:
        p_j=os.path.join(p_i, j)
        Agg_qr=os.listdir(p_j)
        for k in Agg_qr:
            p_k=os.path.join(p_j, k)
            data= open(p_k,'r')
            D=json.load(data)
            for z in D["data"]["transactionData"]:
                Name=z['name']
                count=z['paymentInstruments'][0]['count']
                amount=z['paymentInstruments'][0]['amount']
                clm['Transaction_type'].append(Name)
                clm['Transaction_count'].append(count)
                clm['Transaction_amount'].append(amount)
                clm['State'].append(i)
                clm['Year'].append(j)
                clm['Quarter'].append(int(k.strip('.json')))

df=pd.DataFrame(clm)

df['State'] = df['State'].apply(format_state_name)

create_table_from_dataframe(df, table_name="Aggregated_Transaction")
insert_data_from_dataframe(df, table_name="Aggregated_Transaction")


#Getting Aggregated insurance dataframe from json file and loading it into mysql database as table

path = r"C:\Users\Kumar\Desktop\Guvi\PhonePe Transaction Insights\data\aggregated\insurance\country\india\state"
Agg_ins_list=os.listdir(path)

clm={"State":[], "Year":[], "Quarter":[], "Insurance_type":[], "Insurance_count":[],"Insurance_amount":[] }


for i in Agg_ins_list:
    p_i=os.path.join(path, i)
    Agg_yr=os.listdir(p_i)
    for j in Agg_yr:
        p_j=os.path.join(p_i, j)
        Agg_qr=os.listdir(p_j)
        for k in Agg_qr:
            p_k=os.path.join(p_j, k)
            data= open(p_k,'r')
            D=json.load(data)
            for z in D["data"]["transactionData"]:
                Name=z['name']
                count=z['paymentInstruments'][0]['count']
                amount=z['paymentInstruments'][0]['amount']
                clm['Insurance_type'].append(Name)
                clm['Insurance_count'].append(count)
                clm['Insurance_amount'].append(amount)
                clm['State'].append(i)
                clm['Year'].append(j)
                clm['Quarter'].append(int(k.strip('.json')))

df=pd.DataFrame(clm)

df['State'] = df['State'].apply(format_state_name)

create_table_from_dataframe(df, table_name="Aggregated_Insurance")
insert_data_from_dataframe(df, table_name="Aggregated_Insurance")


#Getting Aggregated user dataframe from json file and loading it into mysql database as table

path = r"C:\Users\Kumar\Desktop\Guvi\PhonePe Transaction Insights\data\aggregated\user\country\india\state"
Agg_user_list=os.listdir(path)

clm={"State":[], "Year":[], "Quarter":[], "Brand":[], "Count":[],"Percentage":[] }

for i in Agg_user_list:
    p_i=os.path.join(path, i)
    Agg_yr=os.listdir(p_i)
    for j in Agg_yr:
        p_j=os.path.join(p_i, j)
        Agg_qr=os.listdir(p_j)
        for k in Agg_qr:
            p_k=os.path.join(p_j, k)
            data= open(p_k,'r')
            D=json.load(data)

            try:
                for z in D["data"]["usersByDevice"]:
                    Brand=z['brand']
                    Count=z['count']
                    Percentage=z['percentage']
                    clm['Brand'].append(Brand)
                    clm['Count'].append(Count)
                    clm['Percentage'].append(Percentage)
                    clm['State'].append(i)
                    clm['Year'].append(j)
                    clm['Quarter'].append(int(k.strip('.json')))
            except:
                pass

df=pd.DataFrame(clm)

df['State'] = df['State'].apply(format_state_name)

create_table_from_dataframe(df, table_name="Aggregated_User")
insert_data_from_dataframe(df, table_name="Aggregated_User")

#Getting Map transaction dataframe from json file and loading it into mysql database as table

path = r"C:\Users\Kumar\Desktop\Guvi\PhonePe Transaction Insights\data\map\transaction\hover\country\india\state"
Map_trans_list=os.listdir(path)

clm={'State':[], 'Year':[],'Quarter':[],'District':[], 'Transaction_count':[], 'Transaction_amount':[]}


for i in Map_trans_list:
    p_i=os.path.join(path, i)
    Map_yr=os.listdir(p_i)
    for j in Map_yr:
        p_j=os.path.join(p_i, j)
        Map_qr=os.listdir(p_j)
        for k in Map_qr:
            p_k=os.path.join(p_j, k)
            data= open(p_k,'r')
            D=json.load(data)
            for z in D["data"]["hoverDataList"]:
                District=z['name']
                count=z['metric'][0]['count']
                amount=z['metric'][0]['amount']
                clm['District'].append(District)
                clm['Transaction_count'].append(count)
                clm['Transaction_amount'].append(amount)
                clm['State'].append(i)
                clm['Year'].append(j)
                clm['Quarter'].append(int(k.strip('.json')))
            
df=pd.DataFrame(clm)

df['State'] = df['State'].apply(format_state_name)

create_table_from_dataframe(df, table_name="Map_Transaction")
insert_data_from_dataframe(df, table_name="Map_Transaction")

#Getting Map user dataframe from json file and loading it into mysql database as table

path = r"C:\Users\Kumar\Desktop\Guvi\PhonePe Transaction Insights\data\map\user\hover\country\india\state"
Map_user_list=os.listdir(path)

clm={'State':[], 'Year':[],'Quarter':[],'District':[], 'Registered_User':[], 'App_Opens':[]}


for i in Map_user_list:
    p_i=os.path.join(path, i)
    Map_yr=os.listdir(p_i)
    for j in Map_yr:
        p_j=os.path.join(p_i, j)
        Map_qr=os.listdir(p_j)
        for k in Map_qr:
            p_k=os.path.join(p_j, k)
            data= open(p_k,'r')
            D=json.load(data)
            for z in D["data"]["hoverData"].items():
                district = z[0]
                registereduser = z[1]["registeredUsers"]
                appopens = z[1]["appOpens"]
                clm['District'].append(district)
                clm['Registered_User'].append(registereduser)
                clm['App_Opens'].append(appopens)
                clm['State'].append(i)
                clm['Year'].append(j)
                clm['Quarter'].append(int(k.strip('.json')))
            
df=pd.DataFrame(clm)

df['State'] = df['State'].apply(format_state_name)

create_table_from_dataframe(df, table_name="Map_User")
insert_data_from_dataframe(df, table_name="Map_User")


#Getting Map insurance dataframe from json file and loading it into mysql database as table

path = r"C:\Users\Kumar\Desktop\Guvi\PhonePe Transaction Insights\data\map\insurance\hover\country\india\state"
Map_ins_list=os.listdir(path)

clm={'State':[], 'Year':[],'Quarter':[],'District':[], 'Transaction_count':[], 'Transaction_amount':[]}


for i in Map_ins_list:
    p_i=os.path.join(path, i)
    Map_yr=os.listdir(p_i)
    for j in Map_yr:
        p_j=os.path.join(p_i, j)
        Map_qr=os.listdir(p_j)
        for k in Map_qr:
            p_k=os.path.join(p_j, k)
            data= open(p_k,'r')
            D=json.load(data)
            for z in D["data"]["hoverDataList"]:
                District=z['name']
                count=z['metric'][0]['count']
                amount=z['metric'][0]['amount']
                clm['District'].append(District)
                clm['Transaction_count'].append(count)
                clm['Transaction_amount'].append(amount)
                clm['State'].append(i)
                clm['Year'].append(j)
                clm['Quarter'].append(int(k.strip('.json')))
            
df=pd.DataFrame(clm)

df['State'] = df['State'].apply(format_state_name)

create_table_from_dataframe(df, table_name="Map_Insurance")
insert_data_from_dataframe(df, table_name="Map_Insurance")

#Getting Top transaction dataframe from json file and loading it into mysql database as table

path = r"C:\Users\Kumar\Desktop\Guvi\PhonePe Transaction Insights\data\top\transaction\country\india\state"
Top_trans_list=os.listdir(path)

clm={'State':[], 'Year':[],'Quarter':[],'pincodes':[], 'Transaction_count':[], 'Transaction_amount':[]}


for i in Top_trans_list:
    p_i=os.path.join(path, i)
    Top_yr=os.listdir(p_i)
    for j in Top_yr:
        p_j=os.path.join(p_i, j)
        Top_qr=os.listdir(p_j)
        for k in Top_qr:
            p_k=os.path.join(p_j, k)
            data= open(p_k,'r')
            D=json.load(data)
            for z in D["data"]["pincodes"]:
                pincodes=z['entityName']
                count=z['metric']['count']
                amount=z['metric']['amount']
                clm['pincodes'].append(pincodes)
                clm['Transaction_count'].append(count)
                clm['Transaction_amount'].append(amount)
                clm['State'].append(i)
                clm['Year'].append(j)
                clm['Quarter'].append(int(k.strip('.json')))
            
df=pd.DataFrame(clm)

df['State'] = df['State'].apply(format_state_name)

create_table_from_dataframe(df, table_name="Top_Transaction")
insert_data_from_dataframe(df, table_name="Top_Transaction")

#Getting Top user dataframe from json file and loading it into mysql database as table

path = r"C:\Users\Kumar\Desktop\Guvi\PhonePe Transaction Insights\data\top\user\country\india\state"
Top_user_list=os.listdir(path)

clm={'State':[], 'Year':[],'Quarter':[],'pincodes':[], 'Registered_Users':[]}


for i in Top_user_list:
    p_i=os.path.join(path, i)
    Top_yr=os.listdir(p_i)
    for j in Top_yr:
        p_j=os.path.join(p_i, j)
        Top_qr=os.listdir(p_j)
        for k in Top_qr:
            p_k=os.path.join(p_j, k)
            data= open(p_k,'r')
            D=json.load(data)
            for z in D["data"]["pincodes"]:
                pincodes=z['name']
                Registered_Users=z['registeredUsers']
                #amount=z['metric']['amount']
                clm['pincodes'].append(pincodes)
                clm['Registered_Users'].append(Registered_Users)
                #clm['Transaction_amount'].append(amount)
                clm['State'].append(i)
                clm['Year'].append(j)
                clm['Quarter'].append(int(k.strip('.json')))
            
df=pd.DataFrame(clm)

df['State'] = df['State'].apply(format_state_name)

create_table_from_dataframe(df, table_name="Top_User")
insert_data_from_dataframe(df, table_name="Top_User")

#Getting Top insurance dataframe from json file and loading it into mysql database as table

path = r"C:\Users\Kumar\Desktop\Guvi\PhonePe Transaction Insights\data\top\insurance\country\india\state"
Top_ins_list=os.listdir(path)

clm={'State':[], 'Year':[],'Quarter':[],'pincodes':[], 'Transaction_count':[], 'Transaction_amount':[]}


for i in Top_ins_list:
    p_i=os.path.join(path, i)
    Top_yr=os.listdir(p_i)
    for j in Top_yr:
        p_j=os.path.join(p_i, j)
        Top_qr=os.listdir(p_j)
        for k in Top_qr:
            p_k=os.path.join(p_j, k)
            data= open(p_k,'r')
            D=json.load(data)
            for z in D["data"]["pincodes"]:
                pincodes=z['entityName']
                count=z['metric']['count']
                amount=z['metric']['amount']
                clm['pincodes'].append(pincodes)
                clm['Transaction_count'].append(count)
                clm['Transaction_amount'].append(amount)
                clm['State'].append(i)
                clm['Year'].append(j)
                clm['Quarter'].append(int(k.strip('.json')))
            
df=pd.DataFrame(clm)

df['State'] = df['State'].apply(format_state_name)

create_table_from_dataframe(df, table_name="Top_Insurance")
insert_data_from_dataframe(df, table_name="Top_Insurance")


